# Решение

## 1. Скачать файл по ссылке

In [1]:
!wget 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv' 'titanic.csv'

--2022-02-10 12:29:38--  https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57018 (56K) [text/plain]
Saving to: ‘titanic.csv’

titanic.csv         100%[===================>]  55.68K  --.-KB/s    in 0.01s   

2022-02-10 12:29:38 (5.08 MB/s) - ‘titanic.csv’ saved [57018/57018]

--2022-02-10 12:29:38--  http://titanic.csv/
Resolving titanic.csv (titanic.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘titanic.csv’
FINISHED --2022-02-10 12:29:38--
Total wall clock time: 0.2s
Downloaded: 1 files, 56K in 0.01s (5.08 MB/s)


## Изучение данных

In [2]:
import pandas as pd

In [135]:
df = pd.read_csv('/content/titanic.csv')

In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


### Пропуски:
* возраст (age, 177 пропусков)
* порт посадки (embarked, 2 пропуска)
* палуба (deck, 688 пропусков)
* город посадки (embark_town, 2 пропуска)

### Заполениние пропусков
#### Возраст 
Принято решение написать небольшую модель, обучить ее на заполненных данных и предугадать примерный возраст в пропущенных ячейках. 

#### Порт посадки 
Все пропуски = U (unknown).

#### Палуба
Для определения палубы будет написана другая модель, но ориентироваться на результаты ее работы я не буду, так как тренировочная выборка слишком мала (203 элемента).

#### Город посадки
Все пропуски = Unknown.

#### Пропуски посадки

In [137]:
def fill_na_embark(data_frame: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame: 
    data_frame['embarked'] = data_frame['embarked'].fillna('U')
    data_frame['embark_town'] = data_frame['embark_town'].fillna('Unknown')

    return data_frame

In [138]:
def for_print_embark(data_frame: pd.core.frame.DataFrame, column_offset=4) -> pd.core.frame.DataFrame:
    for_print = [('Порты:', 'Города:')]
    max_len_port = len(for_print[0][0])
    output_str = ''

    unique_ports, unique_towns = df['embarked'].unique(), df['embark_town'].unique()
    
    for index in range(max(len(unique_ports), len(unique_towns))):
        temp_port = unique_ports[index] if index < len(unique_ports) else ''
        temp_town = unique_towns[index] if index < len(unique_towns) else ''

        max_len_port = max(max_len_port, len(temp_port))

        for_print.append((temp_port, temp_town))

    for port, town in for_print: 
        output_str += f'{port}{" " * (max_len_port + column_offset - len(port))}{town}\n'

    return output_str

In [139]:
df = fill_na_embark(df)
print(for_print_embark(df))

Порты:    Города:
S         Southampton
C         Cherbourg
Q         Queenstown
U         Unknown



#### Пропуски возраста

In [140]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.losses import mae
from tensorflow.keras.metrics import mse

In [170]:
def prepare_data(data_frame): 
    print('----')
    data_frame_copy = data_frame.copy()
    data_frame_copy['sex'] = data_frame_copy['sex'].apply(lambda sex: int(sex == 'female'))
    data_frame_copy['fare'] *= FARE_NORMALIZER
    data_frame_copy['sibsp'] *= SIBSP_NORMALIZER
    data_frame_copy['parch'] *= PARCH_NORMALIZER
    data_frame_copy['class'] = data_frame_copy['class'].apply(lambda cls: CLASSES[cls])
    data_frame_copy['alive'] = data_frame_copy['alive'].apply(lambda alive: int(alive == 'yes'))
    data_frame_copy['alone'] = data_frame_copy['alone'].apply(int)
    
    data_frame_copy['age'] *= AGE_NORMALIZER

    return data_frame_copy

In [171]:
X_COLUMNS = ['pclass', 'sex', 'sibsp', 'parch', 'fare', 'class', 'alive', 'alone']
Y_COLUMNS = ['age']
AGE_NORMALIZER = 0.01
FARE_NORMALIZER = 0.001
SIBSP_NORMALIZER = 0.1
PARCH_NORMALIZER = 0.1
CLASSES = {'First': 0.1, 'Second': 0.4, 'Third': 0.7}


data = df[df.loc[:, 'age'].notna()]
prepare_data(data)

----


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,0,0.22,0.1,0.0,0.007250,S,0.7,man,True,NaN,Southampton,0,0
1,1,1,1,0.38,0.1,0.0,0.071283,C,0.1,woman,False,C,Cherbourg,1,0
2,1,3,1,0.26,0.0,0.0,0.007925,S,0.7,woman,False,NaN,Southampton,1,1
3,1,1,1,0.35,0.1,0.0,0.053100,S,0.1,woman,False,C,Southampton,1,0
4,0,3,0,0.35,0.0,0.0,0.008050,S,0.7,man,True,NaN,Southampton,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,0,3,1,0.39,0.0,0.5,0.029125,Q,0.7,woman,False,NaN,Queenstown,0,0
886,0,2,0,0.27,0.0,0.0,0.013000,S,0.4,man,True,NaN,Southampton,0,1
887,1,1,1,0.19,0.0,0.0,0.030000,S,0.1,woman,False,B,Southampton,1,1
889,1,1,0,0.26,0.0,0.0,0.030000,C,0.1,man,True,C,Cherbourg,1,1
